In [1]:
import os
import torch
from transformers import BarkModel
from transformers import AutoProcessor
from IPython.display import Audio
import nltk
import numpy as np
from bark import generate_audio, SAMPLE_RATE
from scipy.io.wavfile import write as write_wav

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
nltk.download("popular")


c:\Users\jdworzycki\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\jdworzycki\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\jdworzycki\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\jdworzycki\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\jdworzycki\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzip

True

In [2]:
model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

c:\Users\asd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\asd\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [3]:
processor = AutoProcessor.from_pretrained("suno/bark")

In [4]:
script = """
Dzień dobry! Witaj w Małopolskim Centrum Nauki Cogiteon. Informujemy, że rozmowy mogą być nagrywane. Administratorem danych osobowych jest Małopolskie Centrum Nauki Cogiteon z siedzibą w Krakowie. Szczegółowe informacje o przetwarzaniu danych osobowych znajdziesz na naszej stronie internetowej.
Jeśli chcesz dowiedzieć się więcej o naszych aktualnych wystawach, warsztatach lub wydarzeniach, zapraszamy na naszą stronę internetową www.cogiteon.pl.

Cogiteon to miejsce pełne interaktywnych atrakcji, które pozwalają odkrywać, eksperymentować i doświadczyć nauki w nowatorski sposób. Nasza stała wystawa „Człowiek i jego marzenia” oferuje ponad sto interaktywnych stanowisk rozlokowanych na przestrzeni ponad 2,2 tys. m2, a nasze laboratoria zapraszają do udziału w fascynujących warsztatach, takich jak „Laboratorium Smaków” czy „Laboratorium Przyrody”.
Dla dodatkowych informacji, prosimy odwiedzić naszą stronę www.cogiteon.pl lub skontaktować się z nami poprzez e-mail: oferta@cogiteon.pl.
Jeśli potrzebujesz informacji na temat oferty indywidualnej wciśnij 1
Jeśli potrzebujesz informacji na temat oferty dla firm wciśnij 2
Jeśli potrzebujesz innych informacji wciśnij 3

Dziękujemy za telefon i zapraszamy do odkrywania tajemnic nauki z Cogiteon!
""".replace("\n", " ").strip()

In [5]:
sentences = nltk.sent_tokenize(script)
# print(sentences)

['Dzień dobry!', 'Witaj w Małopolskim Centrum Nauki Cogiteon.', 'Informujemy, że rozmowy mogą być nagrywane.', 'Administratorem danych osobowych jest Małopolskie Centrum Nauki Cogiteon z siedzibą w Krakowie.', 'Szczegółowe informacje o przetwarzaniu danych osobowych znajdziesz na naszej stronie internetowej.', 'Jeśli chcesz dowiedzieć się więcej o naszych aktualnych wystawach, warsztatach lub wydarzeniach, zapraszamy na naszą stronę internetową www.cogiteon.pl.', 'Cogiteon to miejsce pełne interaktywnych atrakcji, które pozwalają odkrywać, eksperymentować i doświadczyć nauki w nowatorski sposób.', 'Nasza stała wystawa „Człowiek i jego marzenia” oferuje ponad sto interaktywnych stanowisk rozlokowanych na przestrzeni ponad 2,2 tys.', 'm2, a nasze laboratoria zapraszają do udziału w fascynujących warsztatach, takich jak „Laboratorium Smaków” czy „Laboratorium Przyrody”.', 'Dla dodatkowych informacji, prosimy odwiedzić naszą stronę www.cogiteon.pl lub skontaktować się z nami poprzez e-mail

In [31]:
SPEAKER = "v2/pl_speaker_2"
silence = np.zeros(int(0.1 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]

100%|██████████| 38/38 [00:28<00:00,  1.33it/s]


In [33]:
write_wav(f"{SPEAKER.split('/')[1]}.wav", SAMPLE_RATE, np.concatenate(pieces))